# Demo of Silent Review
This Jupyter notebook is a demo of a post tool reflection component.

This component is a prompt-based approach to identify silent errors in tool calls (errors that do not produce any visible or explicit error message) by determining whether the tool response is relevant, accurate and complete based on the user's query. 

In [ ]:
import random

from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from typing_extensions import Annotated
from langgraph.prebuilt import InjectedState

from altk.post_tool.silent_review.silent_review import (
    SilentReviewForJSONDataComponent,
)
from altk.post_tool.core.toolkit import SilentReviewRunInput, Outcome
from altk.core.toolkit import AgentPhase, ComponentConfig
from altk.core.llm import get_llm

In [ ]:
# Ensure that the following environment variables are set:
# OPENAI_API_KEY = *** openai api key ***
# LLM_PROVIDER = openai.sync
# MODEL_NAME = o4-mini

import os

os.environ["OPENAI_API_KEY"] = "your_api_key_here"

OPENAI_CLIENT = get_llm("openai.sync")
config = ComponentConfig(
    llm_client=OPENAI_CLIENT(
        model_name="o4-mini",
    )
)

Create a get_weather mock tool that returns a temperature given a city. For simplicity, we include the silent review call in the tool function. However, you can integrate the silent review component differently, based on the architecture of your agents and tools. 

In [ ]:
@tool
def get_weather(city: str, state: Annotated[dict, InjectedState]) -> str:
    """Get weather for a given city."""
    if random.random() >= 0.500:
        # Simulates a silent error from an external service
        result = {"weather": "Weather service is under maintenance."}
    else:
        result = {"weather": f"It's sunny and 70F in {city}!"}

    # Use SilentReview component to check if it's a silent error
    review_input = SilentReviewRunInput(
        messages=state["messages"], tool_response=result
    )
    reviewer = SilentReviewForJSONDataComponent()
    review_result = reviewer.process(data=review_input, phase=AgentPhase.RUNTIME)

    if review_result.outcome != Outcome.ACCOMPLISHED:
        # Agent should retry tool call if silent error was detected
        return "Silent error detected, retry the get_weather tool!"
    else:
        return result

Create a simple ReAct agent to demonstrate the silent review component. 

In [ ]:
agent = create_react_agent(
    model="openai:o4-mini", tools=[get_weather], prompt="You are a helpful assistant"
)

Run the agent and print the final result

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
print(result["messages"][-1].content)